In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import re
import string
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.layers import BatchNormalization
from keras.layers import GRU
from keras.preprocessing.sequence import pad_sequences

In [19]:
df = pd.read_json('/kaggle/input/two-sigma-connect-rental-listing-inquiries/train.json.zip')


df.reset_index(drop=True, inplace=True)




df.head(2)

In [20]:
# delete some outliers
df = df.loc[df['price'] <= 20000]
df.shape

In [21]:
df.reset_index(drop=True, inplace=True)

In [22]:
y = df.interest_level.values
df.drop(['interest_level'], axis=1, inplace=True)

In [23]:
df["num_photos"] = df["photos"].apply(len)

# df["num_features"] = df["features"].apply(len)

df.drop(['building_id', 'manager_id', 'street_address', 'display_address', 'listing_id', 'created', 
         'photos'], axis=1, inplace=True)

df2 = df.copy(deep=True)

In [24]:
df2['bathrooms'].replace([10.0, 7.0, 6.5], 6.0, inplace=True)
df2['bedrooms'].replace([8, 7], 6, inplace=True)
df2['num_photos'].replace([68, 60, 50, 44, 43, 46, 45, 44], 38, inplace=True)

In [25]:
def clean_doc2(doc):
    # split into tokens by white space
    tokens = doc.split()
    # convert all words to lowercase
    tokens = [tokens.lower() for tokens in tokens]
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    #tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

def remove_tags(doc):
    text = re.sub('<br\s?\/>|<br>|<p>|<p\s?\/>', "", doc)
    return text

def clean_inter(line):
    text = re.sub("[A-z][?][A-z]", ' ', line)
    return text

df2['description'] = df2['description'].apply(clean_inter)

df2['description'] = df2['description'].apply(remove_tags)

df2['description'] = df2['description'].apply(clean_doc2)

In [26]:
#Some description are 'lorem ipsu' texts
idx_list=[]

for index,row in df2[['description']].iterrows():
    line = row[0]
    #line = str(line)
    if (line.find("lorem ipsu") != -1):
        idx_list.append(index)
print(idx_list)

In [27]:
# replace lorem ipsu text with "missing"
df2['description'].loc[df2.index[idx_list]] = 'missing'

In [28]:
# Empty descriptions
index_list=[]

for index,row in df2[['description']].iterrows():
    line = row[0]
    #line = str(line)
    if (len(str(line)) == 0):
        index_list.append(index)
#print(index_list)
print(len(index_list))

In [29]:
# Replace empty descriptions with "missing"
df2['description'].loc[df2.index[index_list]] = 'missing'

In [30]:
# Empty features
idx_list=[]

for idx,row in df2[['features']].iterrows():
    line = row[0]
    #line = str(line)
    if (len(line) == 0):
        idx_list.append(idx)
#print(idx_list)
print(len(idx_list))

In [31]:
# Replace empty descriptions with "missing"
df2['features'].loc[df2.index[idx_list]] = "[missing]"

In [32]:
df2.head(4)

In [ ]:
#df2['totalwords'] = [len(x.split()) for x in df2['description'].tolist()]

In [33]:
num_vars = ['price', 'latitude', 'longitude']
cat_vars = ['bathrooms', 'bedrooms', 'num_photos']
text_vars = ['description','features']

In [34]:
df2[cat_vars] = df2[cat_vars].astype('category')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df2, y, test_size=0.3, stratify=y, random_state=42)

In [36]:
y_train = pd.get_dummies(y_train)

In [37]:
y_train.head(2)

In [38]:
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])

def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post', truncating='post')
    return padded


In [39]:
# create description tokenizer
tok_desc = create_tokenizer(X_train['description'])

In [40]:
length_desc = 60
# calculate vocabulary size
vocab_size_desc = len(tok_desc.word_index) + 1
print('Vocabulary size: %d' % vocab_size_desc)

In [41]:
# create features tokenizer
tok_feat = create_tokenizer(X_train['features'])

In [42]:
length_feat = 10
# calculate vocabulary size
vocab_size_feat = len(tok_feat.word_index) + 1
print('Vocabulary size: %d' % vocab_size_feat)

In [43]:
def get_tensor(dataset):
    X = {}
#    dictlist = []
    for col in cat_vars:
        X[col] = dataset[col].to_numpy()
#        dictlist.append(X[col].to_numpy())
#        dictlist.append(np.array(X[col]))

    for col in text_vars:
        if col == 'description':
            X[col] = encode_text(tok_desc, dataset['description'], length_desc)
#            dictlist.append(encode_text(tok_desc, X['description'], length_desc))
#           dictlist.append(pad_sequences(dataset[col], maxlen=max_dict[col]))
            
        else:
#            dictlist.append(encode_text(tok_feat, X['features'], length_feat))
            X[col] = encode_text(tok_feat, dataset['features'], length_feat)
#            dictlist.append(pad_sequences(dataset[col], maxlen=max_dict[col]))

    for col in num_vars:
#        dictlist.append(X[col].to_numpy())
        X[col] = dataset[col].to_numpy()
        
        
    return X

In [44]:
X_train_dict = get_tensor(X_train)

In [47]:
#  define model
def get_model():
    inputlayerlist=[]
    # description
    length_desc = 60
    vocab_size_desc = 63940
    # features
    length_feat = 10
    vocab_size_feat = 1000
    input_num={}
    catinput={}
    textinput={}
    embedding={}
    embeddingtxt={}
    embeddingfeat={}
    batchN={}
    flat={}
    
    for col in text_vars:
        if (col == 'description'):
            textinput[col] = Input(shape=(length_desc,), name=col)
            print(col)
            inputlayerlist.append(textinput[col])
                                    
            embeddingtxt[col] = Embedding(vocab_size_desc + 1, 10)(textinput[col])
            embeddingtxt[col] = GRU(16, kernel_regularizer=None)(embeddingtxt[col])
            embeddingtxt[col] = BatchNormalization()(embeddingtxt[col])
            embeddingtxt[col] = Dropout(0.2)(embeddingtxt[col])
        else:
            textinput[col] = Input(shape=(length_feat,), name=col)
            inputlayerlist.append(textinput[col])
            embeddingfeat[col] = Embedding(vocab_size_feat + 1, 10)(textinput[col])
            embeddingfeat[col] = BatchNormalization()(embeddingfeat[col])
            embeddingfeat[col] = Dropout(0.2)(embeddingfeat[col])
            flat[col] = Flatten()(embeddingfeat[col])
    
    for col in num_vars:
        input_num[col] = Input(shape=[1],name=col)
        print(col)
        inputlayerlist.append(input_num[col])
    
    for col in cat_vars:
        n_labels = X_train[col].nunique()
        catinput[col] = Input(shape=[1],name=col)
        print(col)
        inputlayerlist.append(catinput[col])
        embedding[col] = Embedding(n_labels + 1, 5)(catinput[col])
        batchN[col] = BatchNormalization()(embedding[col])
        flat[col] = Flatten()(batchN[col])
    
    
    # num_merged = concatenate(input_num)
    merged = concatenate([embeddingtxt['description'], flat['features'], input_num["price"], input_num["latitude"], 
                          input_num["longitude"], flat['bathrooms'], flat['bedrooms'], flat['num_photos']])
    output = Dense(3, activation='softmax')(merged)
    model = Model(inputs=inputlayerlist, outputs=output)
    # compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [48]:
model = get_model()

In [49]:
model.fit(X_train_dict, y_train, epochs=12, batch_size=16)

In [50]:
# evaluate model on training dataset
_, acc = model.evaluate(X_train_dict, y_train, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))

In [51]:
X_test_dict = get_tensor(X_test)

In [53]:
y_test = pd.get_dummies(y_test)

In [54]:
# evaluate model on test dataset 
_, acc = model.evaluate(X_test_dict, y_test, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))